In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import (
    LoraConfig,
    get_peft_model,
)
import torch
from trl import SFTTrainer, setup_chat_format

c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model = "OpenLLM-Ro/RoLlama3-8b-Instruct"
new_model = "llama-3-8b-chat-aromanian_v2"

In [3]:
torch_dtype = torch.bfloat16
attn_implementation = "eager"

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.90s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

In [6]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [7]:
import pandas as pd
from datasets import Dataset
import unicodedata
df = pd.read_csv("../dataset/nllb_corpus_train.csv")
df_transformed = df.applymap(lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x)  if unicodedata.category(c) != 'Mn']) if type(x) == str else x)
# There are some words that have a -mi at the end, we will eliminate them also
df_transformed.replace(r'\s*-\s*mi\b', '', regex=True, inplace=True)
df_transformed.replace(r'\(i\)', 'i', regex=True, inplace=True)
df_transformed.replace('γ', 'y', regex=True, inplace=True)
df_transformed.replace(r'’', '', regex=True, inplace=True)
df_transformed.replace(r'“', '', regex=True, inplace=True)
df_transformed.replace(r'„', '', regex=True, inplace=True)
df_transformed.columns = [str(q).strip() for q in df_transformed.columns]
# df_transformed.drop(columns=['ro', 'rup', 'translations'], inplace=True)
dataset = Dataset.from_pandas(df_transformed)

dataset = dataset.shuffle(seed=42) # Only use 1000 samples for quick demo


# I'm not sure if apply_chat_template works with llama3, a new jinja template should be created
# def format_chat_template(row, tokenizer=tokenizer):
#     row_json = [{"role": "user", "content": f"Traduce din aromana in romana: {row['rup']}"},
#                {"role": "assistant", "content": row["ro"]}]
#     row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False, add_generation_prompt=True)
#     return row

# dataset = dataset.map(
#     format_chat_template,
#     num_proc=4,
# )


def generate_prompt(data_point):
    begin_of_text = "<|begin_of_text|>"
    end_of_text = "<|end_of_text|>"
    start_header_id = "<|start_header_id|>"
    end_header_id = "<|end_header_id|>"
    eot_id = "<|eot_id|>"
    return {"text":
    f"""{begin_of_text}
        {start_header_id}system{end_header_id} Tradu urmatorul text din aromana in romana:
        {start_header_id}user{end_header_id} {data_point["rup"]}{eot_id}
        {start_header_id} assistant{end_header_id} {data_point["ro"]}{eot_id}
        {end_of_text}"""}

dataset = dataset.map(generate_prompt)

dataset['text'][3]

C:\Users\gheto\AppData\Local\Temp\ipykernel_20940\715056432.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_transformed = df.applymap(lambda x: ''.join([c for c in unicodedata.normalize('NFKD', x)  if unicodedata.category(c) != 'Mn']) if type(x) == str else x)
Map: 100%|██████████| 27033/27033 [00:00<00:00, 30431.02 examples/s]


'<|begin_of_text|>\n        <|start_header_id|>system<|end_header_id|> Tradu această propoziție din aromână în română.\n        <|start_header_id|>user<|end_header_id|> pazitu<|eot_id|>\n        <|start_header_id|> assistant<|end_header_id|> pazit<|eot_id|>\n        <|end_of_text|>'

In [8]:
dataset = dataset.train_test_split(test_size=0.01)
# model.gradient_checkpointing_enable()
# model.gradient_checkpointing_disable()


In [9]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=70,
    warmup_steps=15,
    logging_strategy="steps",
    learning_rate=7e-6,
    fp16=False,
    bf16=False,
    group_by_length=True,
    disable_tqdm=False,
    report_to="none",
)

c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=216,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 271/271 [00:00<00:00, 30104.52 examples/s]


In [11]:
trainer.train()

  1%|          | 70/13381 [00:54<2:42:30,  1.37it/s]

{'loss': 4.4699, 'grad_norm': 3.831062078475952, 'learning_rate': 6.971195570851414e-06, 'epoch': 0.01}


  1%|          | 140/13381 [01:45<2:41:01,  1.37it/s]

{'loss': 2.3875, 'grad_norm': 2.2177364826202393, 'learning_rate': 6.934535388298668e-06, 'epoch': 0.01}


  2%|▏         | 210/13381 [02:37<2:41:00,  1.36it/s]

...

100%|█████████▉| 13370/13381 [2:54:26<00:07,  1.38it/s]

{'loss': 1.5707, 'grad_norm': 6.909970760345459, 'learning_rate': 5.760885829717193e-09, 'epoch': 1.0}


100%|██████████| 13381/13381 [2:54:34<00:00,  1.38it/s]c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\peft\utils\save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
100%|██████████| 13381/13381 [2:54:37<00:00,  1.28it/s]

{'train_runtime': 10477.8373, 'train_samples_per_second': 2.554, 'train_steps_per_second': 1.277, 'train_loss': 1.549549167497072, 'epoch': 1.0}


TrainOutput(global_step=13381, training_loss=1.549549167497072, metrics={'train_runtime': 10477.8373, 'train_samples_per_second': 2.554, 'train_steps_per_second': 1.277, 'total_flos': 6.008832910914355e+16, 'train_loss': 1.549549167497072, 'epoch': 1.0})

In [12]:
begin_of_text = "<|begin_of_text|>"
end_of_text = "<|end_of_text|>"
start_header_id = "<|start_header_id|>"
end_header_id = "<|end_header_id|>"
eot_id = "<|eot_id|>"
prompt =f"""{begin_of_text}
{start_header_id}system{end_header_id} Tradu această propoziție din aromână în română.
{start_header_id}user{end_header_id} Te s-hiba, greaste tata-su al Teatire, - ficiorlu-a meu easte!{eot_id}
{start_header_id}assistant{end_header_id}"""

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


 Se-ncepea, si-l numara tatul lui Teatire - cei 8 copii ai lui!
://
://































In [25]:
def translate(text, src_lang='rup', tgt_lang='ro', max_length=256):
    """Translate a text or list of texts"""
    if isinstance(text, str):
        text = [text]
    
    results = []
    for sentence in text:
        prompt = f"""{begin_of_text}
        {start_header_id}system{end_header_id} Tradu această propoziție din aromână în română.
        {start_header_id}user{end_header_id} {sentence}{eot_id}
        {start_header_id}assistant{end_header_id}"""
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.4, top_k=50, top_p=0.95)
        
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translation = result.split("assistant")[-1].strip()
        results.append(translation)
    
    return results

# Example usage
t = 'Ma, a lor la si paru c-amintara na vasilie-ntreaga'
print(translate(t, 'aromanian', 'romanian'))

['Dar, ei, se pare ca se mai amintesc de o vasilie - de totul!\n        ://']


In [ ]:
from tqdm import tqdm
import sacrebleu
df_ro_rup_test = pd.read_csv("../dataset/nllb_corpus_test.csv")
df_ro_rup_test['ro_pred'] = ''
df_ro_rup_test['rup_pred'] = ''

test_len = len(df_ro_rup_test)
test_len = 200
for i in tqdm(range(0, test_len)):
    rup_texts = df_ro_rup_test.loc[i, 'rup']
    
    if rup_texts:
        df_ro_rup_test.loc[i, 'ro_pred'] = translate(rup_texts, 'romanian', 'aromanian')



In [13]:
# Save the trained model
trainer.model.save_pretrained(new_model)

c:\Users\gheto\miniconda3\envs\aromanianLLM\Lib\site-packages\peft\utils\save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [37]:
import re
bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF()
df_ro_rup_test_t = [re.sub(r'[\n:/]', '', el[0]) for el in df_ro_rup_test['ro_pred'].tolist() if el]

# df_ro_rup_test_t = [el[0] for el in df_ro_rup_test['ro_pred'].tolist() if el]

print("Aromanian to Romanian BLEU:", bleu_calc.corpus_score(df_ro_rup_test['ro'][:200].tolist(), [df_ro_rup_test_t]))

Aromanian to Romanian BLEU: BLEU = 1.32 11.1/2.5/0.4/0.3 (BP = 1.000 ratio = 1.343 hyp_len = 478 ref_len = 356)


In [39]:
df_ro_rup_test_v = []
for ref, pred in zip(df_ro_rup_test['ro'][:200].tolist(), df_ro_rup_test_t):
    if pred:
        df_ro_rup_test_v.append(ref)

print(len(df_ro_rup_test_v))
df_ro_rup_test_t = [el for el in df_ro_rup_test_t if el]
print(len(df_ro_rup_test_t))
print("Aromanian to Romanian BLEU:", bleu_calc.corpus_score(df_ro_rup_test_v, [df_ro_rup_test_t]))
print("Aromanian to Romanian CHRF:", chrf_calc.corpus_score(df_ro_rup_test_v, [df_ro_rup_test_t]))

185
185
Aromanian to Romanian BLEU: BLEU = 0.00 0.0/0.0/0.0/0.0 (BP = 1.000 ratio = 1.261 hyp_len = 449 ref_len = 356)
Aromanian to Romanian CHRF: chrF2 = 6.02
